# Лабораторная работа 1. Перцептрон

## Загрузка модулей

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Загрузка и подготовка данных

In [20]:
df = pd.read_csv('data.csv')

df = df.iloc[np.random.permutation(len(df))]
y = df.iloc[0:100, 4].values
y = np.where(y == "Iris-setosa", 1, -1)
X = df.iloc[0:100, [0, 2]].values
df.head(10)

,0,1,2,3,4
54,6.5,2.8,4.6,1.5,Iris-versicolor
134,6.1,2.6,5.6,1.4,Iris-virginica
44,5.1,3.8,1.9,0.4,Iris-setosa
147,6.5,3.0,5.2,2.0,Iris-virginica
38,4.4,3.0,1.3,0.2,Iris-setosa
10,5.4,3.7,1.5,0.2,Iris-setosa
26,5.0,3.4,1.6,0.4,Iris-setosa
88,5.6,3.0,4.1,1.3,Iris-versicolor
41,4.5,2.3,1.3,0.3,Iris-setosa
120,6.9,3.2,5.7,2.3,Iris-virginica


## Параметры модели

In [21]:
inputSize = X.shape[1] # количество входных сигналов равно количеству признаков задачи 
hiddenSizes = 10 # число нейронов скрытого (А) слоя 
outputSize = 1 if len(y.shape) else y.shape[1] # количество выходных сигналов равно количеству классов задачи


# матрица весов скрытого слоя
Win = np.zeros((1+inputSize,hiddenSizes)) 
# пороги w0 задаем случайными числами
Win[0,:] = (np.random.randint(0, 3, size = (hiddenSizes))) 
# остальные веса задаем случайно -1, 0 или 1 
Win[1:,:] = (np.random.randint(-1, 2, size = (inputSize,hiddenSizes))) 

# случайно инициализируем веса выходного слоя
Wout = np.random.randint(0, 2, size = (1+hiddenSizes,outputSize)).astype(np.float64)

## Функция предсказания

In [22]:
def predict(Xp):
    # выходы первого слоя = входные сигналы * веса первого слоя
    hidden_predict = np.where((np.dot(Xp, Win[1:,:]) + Win[0,:]) >= 0.0, 1, -1).astype(np.float64)
    # выходы второго слоя = выходы первого слоя * веса второго слоя
    out = np.where((np.dot(hidden_predict, Wout[1:,:]) + Wout[0,:]) >= 0.0, 1, -1).astype(np.float64)
    return out, hidden_predict

## Обучение с фиксированным количеством итераций

In [ ]:
n_iter=5
eta = 0.01
for i in range(n_iter):
    for xi, target, j in zip(X, y, range(X.shape[0])):
        pr, hidden = predict(xi) 
        Wout[1:] += ((eta * (target - pr)) * hidden).reshape(-1, 1)
        Wout[0] += eta * (target - pr)

## Обучение с граничным условием о сходимости

In [90]:
eta = 0.01
converged = False  # Переменная для отслеживания сходимости
no_change_count = 0  # Переменная для отслеживания зацикливания

while not converged: # Эпоха
    epoch_weights = []
    for xi, target in zip(X, y):
        pr, hidden = predict(xi)
        Wout[1:] += ((eta * (target - pr)) * hidden).reshape(-1, 1)
        Wout[0] += eta * (target - pr)

    # Проверяем, все ли примеры правильно классифицированы
    all_correct = True
    for xi, target in zip(X, y):
        pr, _ = predict(xi)
        if pr != target:
            all_correct = False
            break

    if all_correct:
        converged = True

    # Если веса не изменились, это может быть признаком зацикливания
    if no_change_count > 0:
        print('Повтор весов')
        break

    weight = Wout.copy()
    weight.sort()
    weight = weight.reshape(1, -1)[0]
    
    epoch_weights.append(weight)
    
    # Сохраняем текущие веса
    for w in epoch_weights:
        if np.array_equal(w, weight):
            print(epoch_weights)
            no_change_count += 1

if converged:
    print('Отсутствуют ошибки')

[array([-0.34, -0.34, -0.22,  0.66,  0.34, -0.34,  0.66,  1.34,  0.66,
        0.66, -0.34])]
Повтор весов


## Проверка модели

In [47]:
y = df.iloc[:, 4].values
y = np.where(y == "Iris-setosa", 1, -1)
X = df.iloc[:, [0, 2]].values
pr, hidden = predict(X)
sum(pr-y.reshape(-1, 1))

array([0.])

In [80]:
Wout.reshape(1, -1)[0].tolist()

[-0.34,
 -0.34,
 -0.21999999999999997,
 0.6599999999999997,
 0.34,
 -0.34,
 0.6599999999999997,
 1.3400000000000003,
 0.6599999999999997,
 0.6599999999999997,
 -0.34]

In [81]:
epoch_weights

[]

In [64]:
epoch_weights

[]

In [79]:
t = [[-0.34, -0.34, -0.22,  0.66,  0.34, -0.34,  0.66,  1.34,  0.66,
        0.66, -0.34]]

[-0.34, -0.34, -0.22,  0.66,  0.34, -0.34,  0.66,  1.34,  0.66, 0.66, -0.34] in t

True